# Imports

In [1]:
import pandas as pd
import numpy as np

---

# Carga de datos

In [2]:
events = pd.read_pickle("../../../data/tp2/events_tp2_formateado.pkl")

---

# Armado de ventanas

In [3]:
for ventana_nro in range(1,6):
    events["ventana_{}".format(ventana_nro)] = (((17+ventana_nro) <= events["date"].dt.day) & ((events["date"].dt.day) <= (19+ventana_nro)))

---

# Armado de validaciones

In [4]:
for validacion_nro in range(1,6):
    events["validacion_{}".format(validacion_nro)] = (((20+validacion_nro) <= events["date"].dt.day) & ((events["date"].dt.day) <= (22+validacion_nro)))

---

# Obtención del tiempo hasta aparición ("objetivo" o "label" o "target") para cada ventana de datos

In [5]:
tres_dias = 3*24*60*60
for validacion_nro in range(1,6):
    validacion = events.loc[events["validacion_{}".format(validacion_nro)]].groupby("ref_hash").agg({"date" : "min"}).reset_index()
    validacion["target"] = validacion["date"] - np.datetime64('2019-04-{} 00:00:00.000000'.format(20+validacion_nro))
    del validacion["date"]
    validacion["target"] = validacion["target"].map(lambda x: (x.delta / 1000000000))
    target = events.loc[events["ventana_{}".format(validacion_nro)]]["ref_hash"].drop_duplicates().to_frame()
    target = target.reset_index()
    del target["index"]
    target = target.merge(validacion, on="ref_hash", how="left").fillna(tres_dias)
    target.to_pickle("../targets/targets_ventana_{}_events.pkl".format(validacion_nro))